In [9]:
%%writefile cuda_bfs.cu
#include<bits/stdc++.h>

using namespace std;

__global__ void bfsKernel(int *vertex, int *edge, int n, int *dist, int *level, int *flag) {
    int t_id = blockIdx.x * blockDim.x + threadIdx.x;
    t_id = t_id + 1;
    //checking for out of scope or impossible conditions
    if(t_id > n or dist[t_id] != *level) return;
    //updating level for the members in distance array
    for(int i=vertex[t_id];i<vertex[t_id+1];i++){
      if(dist[edge[i]]==1000000){
        dist[edge[i]]=*level+1;
        *flag=1;
      }
    }
}
int main(){
  int threadsPerBlock=1024,blocksPerGrid;
  int n;
  cout<<"enter the no.of vertices"<<endl;
  cin>>n;
  blocksPerGrid = (n + 1023)/1024;
  //enter the source vertex
  int src;
  cin>>src;
  vector<vector<int>>edgeList(n);
  //this is the adjacency list representation of the given graph
  //Here the neighbours of each vertex are stored and ending with -1
  int count=0,edgeCount=0;
  while(count<n){
    int t;
    cin>>t;
    if(t==-1){
      count++;
      continue;
    }
    edgeCount++;
    edgeList[count].push_back(t);
  }
  //vertex memory allocation in array
  int *vertex, *dvertex;
  vertex=(int *)malloc(sizeof(int) * (n+2));
  cudaMalloc((void **)&dvertex, sizeof(int) * (n+2));
  //edge memory allocation in array
  //typically it's CSR format
  int *edge, *dedge;
  edge=(int *)malloc(sizeof(int) * (edgeCount+2));
  cudaMalloc((void **)&dedge, sizeof(int) * (edgeCount+2));
  //vertex array keep the index of the starting position of vertex in CSR edge array.
  int k=1;
  for(int i=0;i<=n;i++){
    vertex[i+1]=k;
    if(i==n) break;
    //here all the edges are stored int sequential manner and k is updated here.
    for(int j=0;j<edgeList[i].size();j++){
      edge[k++]=edgeList[i][j];
    }
  }
  for(int i=0;i<edgeCount;i++){
    cout<<i+1<<" "<<edge[i+1]<<endl;
  }
  for(int i=0;i<n;i++){
    cout<<vertex[i+1]<<" ";
  }
  int *dist, *ddist;
  dist=(int *)malloc(sizeof(int) * (n+2));
  cudaMalloc((void **)&ddist,sizeof(int) * (n+2));
  for(int i=1;i<=n;i++){
    dist[i]=1000000;
  }
  dist[src] = 0;
  int *flag, *level;
  int *dflag, *dlevel;
  flag = (int *) (sizeof(int));
  level = (int *) (sizeof(int));
  cudaMalloc((void **)&dflag, sizeof(int));
  cudaMalloc((void **)&dlevel, sizeof(int));
  cudaMemcpy(ddist, dist, sizeof(int) * (n+2), cudaMemcpyHostToDevice);
  cudaMemcpy(dvertex, vertex, sizeof(int) * (n+2), cudaMemcpyHostToDevice);
  cudaMemcpy(dedge, edge, sizeof(int) * (edgeCount + 2), cudaMemcpyHostToDevice);
  int temp_level = 0, temp_flag = 1;
  flag=&temp_flag;
  level=&temp_level;
  while(*flag){
    *flag=0;
    cudaMemcpy(dflag, flag, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dlevel, level, sizeof(int), cudaMemcpyHostToDevice);
    bfsKernel<<<blocksPerGrid,threadsPerBlock>>>(dvertex,dedge,n,ddist,dlevel,dflag);
    *level=*level+1;
    cudaMemcpy(flag, dflag, sizeof(int), cudaMemcpyDeviceToHost);
  }
  cudaMemcpy(dist, ddist, sizeof(int) * (n+2), cudaMemcpyDeviceToHost);
  //After Running parallel BFS 
  cout<<"New distance array is : "<<endl;
  for(int i=1;i<=n;i++){
    cout<<dist[i]<<" ";
  }
 /* vector<pair<int,int>>bfsop(n+2);
  for(int i=1;i<=n;i++){
    bfsop[i]={dist[i],i};
  }
  sort(bfsop.begin(),bfsop.end());
  for(int i=1;i<=n;i++){
    cout<<bfsop[i].second<<" ";
  }*/
  cout<<endl;
  return 0;
}

Overwriting cuda_bfs.cu


In [ ]:
!nvcc cuda_bfs.cu
!./a.out

cuda_bfs.cu